**извлечение признаков из текста на естественном языке**

токенизация и очистка текста


Евгений Борисов borisov.e@solarl.ru

## библиотеки

In [1]:
import re
# import gzip
# import numpy as np
# import numpy.random as rng
import pandas as pd
from tqdm import tqdm

# np.set_printoptions(precision=2) # вывод на печать чисел до 2 знака
pd.options.display.max_colwidth = 200 

tqdm.pandas()

/usr/lib/python3.8/site-packages/tqdm/std.py:703: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
# from sklearn import __version__ as SKLEARN_VERSION

# from sklearn.feature_extraction.text import HashingVectorizer

# from sklearn.model_selection import train_test_split
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import classification_report
# from sklearn.metrics import accuracy_score

# SKLEARN_VERSION

## тексты

In [3]:
# загружаем тексты
data = pd.read_pickle('../data/news.pkl.gz')
print('записей:',len(data))
data.sample(5)

записей: 3196


,text,tag
2657,"Например, исследователи из Эдинбургского университета в Великобритании предположили, что инопланетные цивилизации могут размещаться на коричневых карликах, потому как их температура и давление оче...",science
624,Как выбрать маникюр для Нового Года: тренды 2017\n\n1 декабря 2016 в 9:00\n\nBLOG.TAM.BY\n\nВ преддверии новогодних праздников каждая девушка мечтает выглядеть потрясающе. И это касается не только...,woman
135,"НХЛ: Семейка Адамсов взгрела ""Анахайм""В регулярном чемпионате НХЛ\nзавершились шесть матчей. ""Детройт"" на своем льду обыграл ""Нью-Джерси"",\nпрервав серию из трех домашних поражений, чего не случал...",sport
2920,"По неофициальной информации, ноу-хау планируют назвать Beijing BJ80 Sport. От оригинала новую модель будут отличать воздухозаборник из пластика на капоте, и 20-дюймовые колесные диски, а также тор...",auto
1393,"Москва, 13 декабря. Опрос, проведенный социологическим «Аналитическим центром Юрия Левады» показал, что три четверти россиян уверены в том, что президент Владимир Путин последовательно работает на...",politics


In [4]:
# длина строк
pd.DataFrame(data['text'].str.len()).describe([.1,.25,.5,.75,.95]).astype(int).T

,count,mean,std,min,10%,25%,50%,75%,95%,max
text,3196,1720,2177,25,370,675,1070,1813,5761,30710


In [5]:
# количество категорий
data['tag'].drop_duplicates().count()

13

## токенизация и очистка

In [6]:
# применяет список замен pat к строке s
def replace_patterns(s,pat):
    if len(pat)<1: return s
    return  replace_patterns( re.sub(pat[0][0],pat[0][1],s), pat[1:] )

# нормализация текста
def text_normalizer(s):
    pat = [
       [r'ё','е'] # замена ё для унификации
       ,[r'</?[a-z]+>',' '] # удаляем xml
       ,[r'\d+',' '] # удаляем цифры
       ,[r'[\W]+',' '] # удаляем спец.символы
       ,[r' +',' '] # удаляем повторы пробелов
    ]
    return replace_patterns(s.lower(),pat).strip()

# разрезаем стоки на слова
def tokenize(s): 
    return [ w for w in s.split() if (len(w)>1) ]

In [7]:
data['ctext'] = data['text'].progress_apply(text_normalizer).progress_apply( tokenize )

100%|██████████| 3196/3196 [00:00<00:00, 34385.36it/s]


In [8]:
# собираем словарь
vcb = [ set(s) for s in data['ctext'].tolist() ]
vcb = sorted(set.union(*vcb))
print('словарь %i слов'%(len(vcb)))
# pd.DataFrame( vcb0 ).to_csv('voc0.txt',index=False,header=False)

словарь 81942 слов


In [9]:
data.sample(5)

,text,tag,ctext
3078,Новости. Холмск - События в городе и районе На Сахалине готовятся\nк празднованию Дня коренных народов мира Представители этносов севера\nСахалинской области готовятся к празднованию Дня коренных ...,social,"[новости, холмск, события, городе, районе, на, сахалине, готовятся, празднованию, дня, коренных, народов, мира, представители, этносов, севера, сахалинской, области, готовятся, празднованию, дня, ..."
2143,"Футбол\n\nКриштиану Роналду: «Я один из тех игроков, которых можно пересчитать по пальцам одной руки»",sport,"[футбол, криштиану, роналду, один, из, тех, игроков, которых, можно, пересчитать, по, пальцам, одной, руки]"
72,Установка Gentoo на уже существующий раздел дискаDmitry G Zimin написал\nстатью в которой показал как вручную установить Gentoo Linux на уже\nсуществующий раздел диска.,tech,"[установка, gentoo, на, уже, существующий, раздел, дискаdmitry, zimin, написал, статью, которой, показал, как, вручную, установить, gentoo, linux, на, уже, существующий, раздел, диска]"
1179,В сквере Котовка снова рубят. Защитники сквера остановили людей с пилами\n\n6 декабря 2016 в 11:48\n\nЕлена Толкачева / REALTY.TUT.BY / Фото: Евгений Ерчак\n\nВо вторник утром в сквер Котовка прие...,realty,"[сквере, котовка, снова, рубят, защитники, сквера, остановили, людей, пилами, декабря, елена, толкачева, realty, tut, by, фото, евгений, ерчак, во, вторник, утром, сквер, котовка, приехала, техник..."
434,"Коалиция США нанесла удары по госпиталю в Мосуле\n\n8 декабря 2016 в 9:01\n\nLenta.ru\n\nВозглавляемая США коалиция нанесла удары по госпиталю в Мосуле, который использовали боевики «Исламского го...",politics,"[коалиция, сша, нанесла, удары, по, госпиталю, мосуле, декабря, lenta, ru, возглавляемая, сша, коалиция, нанесла, удары, по, госпиталю, мосуле, который, использовали, боевики, исламского, государс..."


In [10]:
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords as nltk_stopwords
# from nltk import download as nltk_download
# nltk_download('stopwords')

stopwords = set(nltk_stopwords.words('russian'))
stemmer = SnowballStemmer('russian')

# выкидываем stopwords, выполняем стеминг
def stem(s): return [ stemmer.stem(w) for w in s if w not in stopwords ]

data['ctext'] = data['ctext'].progress_apply( stem )

100%|██████████| 3196/3196 [00:14<00:00, 213.80it/s]


In [11]:
# собираем словарь
vcb = [ set(s) for s in data['ctext'].tolist() ]
vcb = sorted(set.union(*vcb))
print('словарь %i слов'%(len(vcb)))
# pd.DataFrame( vcb1 ).to_csv('voc0.txt',index=False,header=False)

словарь 35002 слов


In [12]:
data.sample(3)

,text,tag,ctext
2220,"Автобус попал в ДТП на Солидарности: водителю стало плохо за рулем\n\nДТП с участием общественного транспорта произошло утром во вторник, 13 декабря, в Невском районе Петербурга. Никто из людей не...",incident,"[автобус, попа, дтп, солидарн, водител, стал, плох, рул, дтп, участ, обществен, транспорт, произошл, утр, вторник, декабр, невск, район, петербург, никт, люд, пострада, вконтакт, facebook, twitter..."
506,"В Гродно три человека госпитализировали после неудачного поворота налево\n\n10 декабря 2016 в 12:33\n\nAUTO.TUT.BY\n\nТри человека, в том числе 14-летняя девочка, госпитализированы после ДТП, кото...",incident,"[гродн, человек, госпитализирова, неудачн, поворот, налев, декабр, auto, tut, by, человек, числ, летн, девочк, госпитализирова, дтп, котор, произошл, пятниц, декабр, гродн, фот, игор, ремзик, tut,..."
1897,"Пассажиры экстренно севшего в Нью-Йорке самолёта Lufthansa эвакуированы. Сейчас экстренные службы проверяют авиалайнер на наличие взрывчатки на его борту.\n\nРанее сообщалось, что самолёт компании...",incident,"[пассажир, экстрен, севш, нью, йорк, самолет, lufthansa, эвакуирова, экстрен, служб, проверя, авиалайнер, налич, взрывчатк, борт, ран, сообща, самолет, компан, lufthansa, летевш, хьюстон, франкфур..."
